In [1]:
# Célula 0 — Instalar dependências
! pip install pandas matplotlib openpyxl os



ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
# Célula 1 — Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
# Célula 2 — Conferir diretório atual
os.getcwd()
os.listdir()

'e:\\github\\dev\\dev\\Bacharelado - Ciência da Computação IMPACTA\\5 Semestre\\Inteligência Computacional\\projetofinal-testes'

In [ ]:
# Célula 3 — Montar caminho do arquivo com os
caminho = os.path.join("br_seeg_emissoes_brasil.xlsx")
df = pd.read_excel(caminho)


In [ ]:
# Célula 4 — Inspeção inicial
df.head()
df.columns
df.info()


In [ ]:
# Célula 5 — Ajustar tipos
df["ano"] = df["ano"].astype(int)
df["emissao"] = pd.to_numeric(df["emissao"], errors="coerce")

In [ ]:
# Célula 6 — Filtros 